# Spam Filter (Exercise 3.4)

A spam/ham classifier based on the provided solutions.

In [6]:
import tarfile

## Loading Data

In [7]:
def fetch_spam_data():
    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
                 
    files = (("easy_ham_1", "20030228_easy_ham"),
             ("easy_ham_2", "20030228_easy_ham_2"), 
             ("hard_ham_1", "20030228_hard_ham"),
             ("spam_1", "20030228_spam"),
             ("spam_2", "20030228_spam_2"))
    
    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
                 
    for name, filename in files:
        url = spam_root + filename + ".tar.bz2"
        if not (spam_path / name).is_dir():
            path = (spam_path / name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / name for name in [f[0] for f in files]]

In [ ]:
print(fetch_spam_data())